<a href="https://colab.research.google.com/github/williambiondi/RecommenderGS/blob/main/recommenderGranSasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importazione dei dati e librerie


In [1]:
!pip install lightfm -q
!pip install altair -q
!pip install altair_viewer -q
!pip install requests -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 7.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import altair as alt
import requests
from datetime import datetime
import re
from scipy.sparse.linalg._dsolve.linsolve import csr_matrix
from ast import literal_eval

alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

In [3]:
items_df = pd.read_excel("OrdiniGranSasso.xlsx")
purchases_df = pd.read_excel("OrdiniGranSassoDescrizione.xlsx")

Pulizia dei dati


In [4]:
purchases_df.head()

,Id,PaymentMethodSystemName,CustomerId,CreatedOnUtc,OrderSubtotalExclTax,OrderSubtotalInclTax,OrderShippingExclTax,CustomerCurrencyCode,CurrencyRate,OrderShippingInclTax,...,Sku,DescrizioneEstesa,Categoria,DescrizioneEstesaInglese,UnitPriceExclTax,UnitPriceInclTax,Email,CountryId,Name,NumericIsoCode
0,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,A20205716614238589,Maglia girocollo in pura lana merino con lavor...,NaN,Crew neck sweater in pure merino wool with mic...,106.557,129.9995,lovagliom@hotmail.it,45,Italy,380
1,8813,FoxNetSoft.StripeDirect,9165140,2021-01-01 09:39:57.3829465,278.689,340.0006,12.0,EUR,1.0,12.0,...,A20201312414230005,Maglia dolcevita in pura lana merino punto per...,NaN,"Turtleneck sweater in pure merino wool, embell...",131.148,160.0006,aaron.fandrei@icloud.com,35,Germany,276
2,8813,FoxNetSoft.StripeDirect,9165140,2021-01-01 09:39:57.3829465,278.689,340.0006,12.0,EUR,1.0,12.0,...,A20205711414241046,Cardigan in pura lana merino extra fine con zi...,NaN,Cardigan in pure extra fine merino wool with t...,147.541,180.0000,aaron.fandrei@icloud.com,35,Germany,276
3,8814,Payments.CheckMoneyOrder,9489715,2021-01-01 10:23:40.5132880,237.704,289.9989,0.0,EUR,1.0,0.0,...,A20201312022612852,Maglia girocollo in Air Wool a 3 fili effetto ...,NaN,3-ply moulinè effect Air Wool crew neck sweate...,118.852,144.9994,vtpagone@gmail.com,45,Italy,380
4,8814,Payments.CheckMoneyOrder,9489715,2021-01-01 10:23:40.5132880,237.704,289.9989,0.0,EUR,1.0,0.0,...,A20201314022615116,<p>Maglia con scollo a V in Air Wool con lavor...,NaN,<p>Argyle Air Wool V-neck sweater. The special...,118.852,144.9994,vtpagone@gmail.com,45,Italy,380


In [5]:
purchases_df.dropna(subset=['DescrizioneEstesaInglese'], inplace=True)

In [6]:
currencies = purchases_df['CustomerCurrencyCode']
currencies.drop_duplicates(inplace=True)

In [7]:
currencies

0       EUR
7       NOK
11      USD
13      CAD
60      CHF
102     GBP
326     TRY
850     AUD
1933    JPY
Name: CustomerCurrencyCode, dtype: object

In [8]:
def exchange_rate(currency):
  KEY = 'MY_KEY'
  url = f'http://data.fixer.io/api/latest?access_key={KEY}'
  try:
    response = requests.get(url)
    data = response.json()
    exchange_rate = float(data['rates'][currency])
    return 1/exchange_rate
  except requests.RequestException as e:
    print("error")
    return None

In [9]:
rates = {}
for currency in currencies:
  rates[currency] = exchange_rate(currency)

In [ ]:
rates

{'EUR': 1.0,
 'NOK': 0.08527916219022132,
 'USD': 0.9150646172879499,
 'CAD': 0.672093146733392,
 'CHF': 1.0366013573258173,
 'GBP': 1.1523246421167743,
 'TRY': 0.031690235804192494,
 'AUD': 0.6017926198560272,
 'JPY': 0.006118338554886612}

In [10]:
def to_euro(row, column):
  rate = rates.get(row['CustomerCurrencyCode'], 1)
  return round(row[column] * rate,2)

In [11]:
purchases_df['UnitPriceTaxEUR'] = purchases_df.apply(lambda x: to_euro(x, 'UnitPriceInclTax'), axis=1)
purchases_df['RefundedEUR'] = purchases_df.apply(lambda x: to_euro(x, 'RefundedAmount'), axis=1)

In [12]:
purchases_df.head()

,Id,PaymentMethodSystemName,CustomerId,CreatedOnUtc,OrderSubtotalExclTax,OrderSubtotalInclTax,OrderShippingExclTax,CustomerCurrencyCode,CurrencyRate,OrderShippingInclTax,...,Categoria,DescrizioneEstesaInglese,UnitPriceExclTax,UnitPriceInclTax,Email,CountryId,Name,NumericIsoCode,UnitPriceTaxEUR,RefundedEUR
0,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,NaN,Crew neck sweater in pure merino wool with mic...,106.557,129.9995,lovagliom@hotmail.it,45,Italy,380,130.0,0.0
1,8813,FoxNetSoft.StripeDirect,9165140,2021-01-01 09:39:57.3829465,278.689,340.0006,12.0,EUR,1.0,12.0,...,NaN,"Turtleneck sweater in pure merino wool, embell...",131.148,160.0006,aaron.fandrei@icloud.com,35,Germany,276,160.0,0.0
2,8813,FoxNetSoft.StripeDirect,9165140,2021-01-01 09:39:57.3829465,278.689,340.0006,12.0,EUR,1.0,12.0,...,NaN,Cardigan in pure extra fine merino wool with t...,147.541,180.0000,aaron.fandrei@icloud.com,35,Germany,276,180.0,0.0
3,8814,Payments.CheckMoneyOrder,9489715,2021-01-01 10:23:40.5132880,237.704,289.9989,0.0,EUR,1.0,0.0,...,NaN,3-ply moulinè effect Air Wool crew neck sweate...,118.852,144.9994,vtpagone@gmail.com,45,Italy,380,145.0,0.0
4,8814,Payments.CheckMoneyOrder,9489715,2021-01-01 10:23:40.5132880,237.704,289.9989,0.0,EUR,1.0,0.0,...,NaN,<p>Argyle Air Wool V-neck sweater. The special...,118.852,144.9994,vtpagone@gmail.com,45,Italy,380,145.0,0.0


In [13]:
purchases_selected = purchases_df[['Id', 'CustomerId', 'CreatedOnUtc', 'RefundedEUR','UnitPriceTaxEUR','Sku']]

In [16]:
customers_selected = purchases_df[['CustomerId', 'Name']]
customers_selected.rename(columns={'Name':'Country'}, inplace=True)

<ipython-input-16-5eea51df86c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_selected.rename(columns={'Name':'Country'}, inplace=True)


,CustomerId,Country
0,9486139,Italy
1,9165140,Germany
2,9165140,Germany
3,9489715,Italy
4,9489715,Italy


In [18]:
items_df = pd.merge(purchases_df, items_df, on='Sku', how='inner')

In [19]:
items_df.head()

,Id_x,PaymentMethodSystemName,CustomerId_x,CreatedOnUtc_x,OrderSubtotalExclTax,OrderSubtotalInclTax,OrderShippingExclTax,CustomerCurrencyCode,CurrencyRate,OrderShippingInclTax,...,NumericIsoCode,UnitPriceTaxEUR,RefundedEUR,Id_y,CustomerId_y,CreatedOnUtc_y,RefundedAmount_y,LocaleValue,UnitPriceInclTax_y,Country
0,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8812,9486139,2021-01-01 09:29:05.2513839,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
1,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8860,9568742,2021-01-03 23:11:34.9635880,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
2,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,9311,9862927,2021-02-13 12:39:33.7391675,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
3,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,9772,8811811,2021-04-10 11:33:22.9952881,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Germany
4,8860,FoxNetSoft.PayPalStandardPro,9568742,2021-01-03 23:11:34.9635880,459.016,559.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8812,9486139,2021-01-01 09:29:05.2513839,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy


In [20]:
items_df.rename(columns={'Sku_x':'Sku', 'UnitPriceTaxEUR':'UnitPrice'}, inplace=True)

In [21]:
items_df.head()

,Id_x,PaymentMethodSystemName,CustomerId_x,CreatedOnUtc_x,OrderSubtotalExclTax,OrderSubtotalInclTax,OrderShippingExclTax,CustomerCurrencyCode,CurrencyRate,OrderShippingInclTax,...,NumericIsoCode,UnitPrice,RefundedEUR,Id_y,CustomerId_y,CreatedOnUtc_y,RefundedAmount_y,LocaleValue,UnitPriceInclTax_y,Country
0,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8812,9486139,2021-01-01 09:29:05.2513839,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
1,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8860,9568742,2021-01-03 23:11:34.9635880,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
2,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,9311,9862927,2021-02-13 12:39:33.7391675,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy
3,8812,FoxNetSoft.StripeDirect,9486139,2021-01-01 09:29:05.2513839,106.557,129.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,9772,8811811,2021-04-10 11:33:22.9952881,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Germany
4,8860,FoxNetSoft.PayPalStandardPro,9568742,2021-01-03 23:11:34.9635880,459.016,559.9995,0.0,EUR,1.0,0.0,...,380,130.0,0.0,8812,9486139,2021-01-01 09:29:05.2513839,0.0,MICRO RHOMBUS STITCH CREW NECK,129.9995,Italy


In [23]:
items_selected = items_df[['Sku', 'LocaleValue', 'Categoria','DescrizioneEstesaInglese','UnitPrice']].drop_duplicates('Sku')

In [27]:
items_selected['Collection'] = ['Fall-Winter' if sku[0]=='A' else 'Spring-Summer' for sku in items_selected['Sku']]
items_selected['Availability'] = [1 if sku[1:5]=='2023' else 0 for sku in items_selected['Sku']]
items_selected['Color'] = [sku[15:18] for sku in items_selected['Sku']]
items_selected['Gender'] = ['Women' if int(sku[7]) == 2 else 'Men' for sku in items_selected['Sku']]

In [ ]:
items_selected.dropna(subset=['DescrizioneEstesaInglese'], inplace=True)

Recupero categorie e composizione dalla descrizione

In [30]:
def clean_description(description):
  clean = re.sub(r'<[^>]+>','', description.lower().strip())
  return re.sub(r'[^\w\s]','',clean)

In [31]:
def get_composition(description, composition_list):
  for word in description.split(" "):
    if word.lower() in composition_list:
      return word.lower()

In [28]:
composition_list = ['alcantara', 'cashmere','wool', 'cotton', 'nylon', 'viscose', 'eco-feather', 'silk']

In [29]:
category_set = set()
for category in items_selected['Categoria']:
  if not pd.isna(category):
    for item in category.split(";"):
      category_set.add(item)
  else:
    continue
category_list = list(category_set)

In [32]:
items_selected['Description'] = items_selected['DescrizioneEstesaInglese'].apply(clean_description)
items_selected.drop(columns=['DescrizioneEstesaInglese'], inplace=True)
items_selected['Composition'] = items_selected['Description'].apply(lambda x: get_composition(x, composition_list))

In [33]:
categories = {
    "Pantaloni e Bermuda": {"pant", "bermuda", "trousers", "shorts"},
    "Cashmere": {"cashmere"},
    "Beachwear": {"beachwear", "swimwear", "bikini", "swimsuit"},
    "Maglieria": {"knit", "sweater", "cardigan", "wool"},
    "Pantaloni e Gonne": {"pant", "skirt"},
    "I classici": {"classic", "traditional", "timeless"},
    "Polo, Camicie e T-shirt": {"polo", "shirt", "t-shirt", "tee"},
    "Abiti, Pantaloni e Gonne": {"dress", "pant", "skirt"},
    "Sciarpe e Cappelli": {"scarf", "hat", "cap"},
    "Capispalla": {"coat", "jacket", "outerwear"},
    "Vintage": {"vintage", "retro", "old-school"},
    "Pantaloni": {"pant", "trousers", "slacks"},
    "Giacche": {"jacket", "cardigan"},
    "Polo, Camicie e Top": {"polo", "shirt", "top"}
}

In [34]:
def get_category(description, categories):
  description = set(description.split(" ")[:10])
  matched = list()
  for category, keywords in categories.items():
    if description.intersection(keywords):
      matched.append(category)
  return matched

In [35]:
for idx, row in items_selected.iterrows():
  if pd.isna(row['Categoria']):
    predicted = get_category(row['Description'], categories)
    if predicted:
      items_selected.at[idx,'Categoria'] = predicted
  else:
    items_selected.at[idx,'Categoria'] = row['Categoria'].split(";")

In [36]:
items_selected.rename(columns={'LocaleValue':'ItemName','Categoria':'Category'}, inplace=True)
items_selected.head()

,Sku,ItemName,Category,UnitPrice,Collection,Availability,Color,Gender,Description,Composition
0,A20205716614238589,MICRO RHOMBUS STITCH CREW NECK,[Maglieria],130.0,Fall-Winter,0,589,Men,crew neck sweater in pure merino wool with mic...,wool
16,A20201312414230005,WAVE PATTERN TURTLENECK,[Maglieria],160.0,Fall-Winter,0,005,Men,turtleneck sweater in pure merino wool embelli...,wool
20,A20205711414241046,SALT AND PEPPER STITCH FULL ZIP,"[Maglieria, Giacche]",180.0,Fall-Winter,0,046,Men,cardigan in pure extra fine merino wool with t...,wool
29,A20201312022612852,AIR WOOL ARGYLE CREW NECK,[Maglieria],145.0,Fall-Winter,0,852,Men,3ply moulinè effect air wool crew neck sweater...,wool
38,A20201314022615116,AIR WOOL ARGYLE V NECK,[Maglieria],145.0,Fall-Winter,0,116,Men,argyle air wool vneck sweater the special feat...,wool


In [37]:
items_selected.to_csv('ArticoliGransasso.csv')
purchases_selected.to_csv('OrdiniGransasso.csv')
customers_selected.to_csv('ClientiGransasso.csv')

#Analitiche

In [39]:
items_selected = pd.read_csv('ArticoliGransasso.csv')
customers_selected = pd.read_csv('ClientiGransasso.csv')
purchases_selected = pd.read_csv('OrdiniGransasso.csv')

In [40]:
items_selected.drop(columns=['Unnamed: 0'], inplace=True)
customers_selected.drop(columns=['Unnamed: 0'], inplace=True)
purchases_selected.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
purchases_selected.head()

,Id,CustomerId,CreatedOnUtc,RefundedEUR,UnitPriceTaxEUR,Sku
0,8812,9486139,2021-01-01 09:29:05.2513839,0.0,130.0,A20205716614238589
1,8813,9165140,2021-01-01 09:39:57.3829465,0.0,160.0,A20201312414230005
2,8813,9165140,2021-01-01 09:39:57.3829465,0.0,180.0,A20205711414241046
3,8814,9489715,2021-01-01 10:23:40.5132880,0.0,145.0,A20201312022612852
4,8814,9489715,2021-01-01 10:23:40.5132880,0.0,145.0,A20201314022615116


In [ ]:
customers_selected.head()

,CustomerId,Name
0,9486139,Italy
1,9165140,Germany
2,9165140,Germany
3,9489715,Italy
4,9489715,Italy


In [ ]:
items_selected.describe()

,UnitPrice,Color
count,3497.000000,3497.000000
mean,243.620083,330.451244
std,153.449605,274.878933
min,0.000000,1.000000
25%,150.000000,93.000000
50%,204.670000,241.000000
75%,280.000000,598.000000
max,2095.500000,997.000000


In [ ]:
price_chart = alt.Chart(items_selected).mark_bar().encode(
    x='UnitPrice:Q',
    y='count()'
).encode(
    alt.X('UnitPrice:Q', bin=alt.Bin(step=150), title='Prezzo in EUR'),
    alt.Y('count()', title='Numero Articoli')
).properties(title='Distribuzione prezzi')
alt.hconcat(price_chart)

alt.HConcatChart(...)

In [ ]:
total_per_order = purchases_selected.merge(items_selected, on='Sku').groupby('Id')['UnitPrice'].sum().reset_index()

In [ ]:
total_per_order.describe()

,Id,UnitPrice
count,10777.000000,10777.000000
mean,14231.242739,444.062035
std,3129.047603,506.253430
min,8812.000000,0.000000
25%,11525.000000,200.000000
50%,14228.000000,305.290000
75%,16945.000000,524.910000
max,19650.000000,16887.750000


In [ ]:
orders_chart = alt.Chart(total_per_order).mark_bar().encode(
    x='UnitPrice:Q',
    y='count()'
).encode(alt.X('UnitPrice:Q', bin=alt.Bin(step=500), title='Prezzo in EUR'),
         alt.Y('count()', title='Numero Ordini')).properties(title='Distribuzione spese per singolo ordine')
alt.hconcat(orders_chart)

alt.HConcatChart(...)

In [ ]:
order_customer = purchases_selected.merge(customers_selected, on='CustomerId')
full = order_customer.merge(items_selected, on='Sku')
full.head()
orders = full.groupby(['Id','Country'])['UnitPrice'].sum().reset_index()
mean_order_per_country = full.groupby(['Country'])['UnitPrice'].mean().reset_index()

In [ ]:
nations_filter = alt.selection_multi(fields=["Country"])
nations_chart = alt.Chart(orders).mark_bar().encode(
    x="count()",
    y=alt.Y("Country:N"),
    color=alt.condition(nations_filter,
        alt.Color("Name:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(title='Ordini per nazione',selection=nations_filter)
nations_chart_mean_expense = alt.Chart(mean_order_per_country).mark_bar().encode(
    x="UnitPrice:Q",
    y=alt.Y("Country:N"),
    color=alt.condition(nations_filter,
        alt.Color("Country:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(title='Spesa media per singolo ordine per nazione',selection=nations_filter)
alt.hconcat(nations_chart,nations_chart_mean_expense)

alt.HConcatChart(...)

#Preparazione dati per modello


In [41]:
item_dict = dict()
for i in range(items_selected.shape[0]):
  item_dict[''] = items_selected.loc[i,'Sku']

In [42]:
bins = list(range(0, int(items_selected['UnitPrice'].max()) + 150, 150))
items_selected['PriceRange'] = pd.cut(items_selected['UnitPrice'], bins = bins)

In [43]:
items_selected['Category'] = items_selected['Category'].fillna('[]')
items_selected['Category'] = items_selected['Category'].apply(literal_eval)
expanded = items_selected.explode('Category')
dummies_cat = pd.get_dummies(expanded['Category'])
df_dummies_cat = expanded[['Sku']].join(dummies_cat).groupby('Sku').max()
items_selected_dummies = pd.get_dummies(items_selected, columns=['Collection','Composition','Gender','PriceRange'])
items_selected_dummies.drop_duplicates(subset=['Sku'], inplace=True)
items_selected = df_dummies_cat.merge(items_selected_dummies, on='Sku')

In [ ]:
items_selected.columns

Index(['Sku', 'Abiti, Pantaloni e Gonne', 'Beachwear', 'Capispalla',
       'Cashmere', 'Giacche', 'I classici', 'Maglieria', 'Maglieria, Giacche',
       'Pantaloni', 'Pantaloni e Bermuda', 'Pantaloni e Gonne',
       'Polo, Camicie e T-shirt', 'Polo, Camicie e Top', 'Sciarpe e Cappelli',
       'Vintage', 'ItemName', 'Category', 'UnitPrice', 'Color', 'Description',
       'Collection_Fall-Winter', 'Collection_Spring-Summer',
       'Composition_alcantara', 'Composition_cashmere', 'Composition_cotton',
       'Composition_nylon', 'Composition_silk', 'Composition_viscose',
       'Composition_wool', 'Gender_Men', 'Gender_Women', 'PriceRange_(0, 150]',
       'PriceRange_(150, 300]', 'PriceRange_(300, 450]',
       'PriceRange_(450, 600]', 'PriceRange_(600, 750]',
       'PriceRange_(750, 900]', 'PriceRange_(900, 1050]',
       'PriceRange_(1050, 1200]', 'PriceRange_(1200, 1350]',
       'PriceRange_(1350, 1500]', 'PriceRange_(1500, 1650]',
       'PriceRange_(1650, 1800]', 'PriceRange_

In [44]:
items_selected_dummies = items_selected.drop(['Sku','ItemName','Category','UnitPrice','Description'],
                                             axis=1)
items_selected_csr = csr_matrix(items_selected_dummies)

In [45]:
interaction_matrix = pd.pivot_table(purchases_selected,
                                    index='CustomerId',
                                    columns='Sku',
                                    values='Id',
                                    aggfunc='count',
                                    fill_value=0)

In [46]:
interaction_matrix

Sku,A20195423112830018,A20195423112830099,A20195423112830224,A20195423112830546,A20195423112830564,A20195427012500099,A20195427012500463,A20195427012500553,A20195427012500598,A20195427212500099,...,P20239010141300664,P20239010141400099,P20239010141400157,P20239010141400412,P20239010141400597,P20239010141600597,P20239010141600664,P20239010141700100,P20239010141700101,P20239710176002005
CustomerId,,,,,,,,,,,,,,,,,,,,,
10609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80815,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25692342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25721539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25788378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
items_selected

,Sku,"Abiti, Pantaloni e Gonne",Beachwear,Capispalla,Cashmere,Giacche,I classici,Maglieria,"Maglieria, Giacche",Pantaloni,...,"PriceRange_(600, 750]","PriceRange_(750, 900]","PriceRange_(900, 1050]","PriceRange_(1050, 1200]","PriceRange_(1200, 1350]","PriceRange_(1350, 1500]","PriceRange_(1500, 1650]","PriceRange_(1650, 1800]","PriceRange_(1800, 1950]","PriceRange_(1950, 2100]"
0,A20195423112830018,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A20195423112830099,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A20195423112830224,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A20195423112830546,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A20195423112830564,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3492,P20239010141600597,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3493,P20239010141600664,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3494,P20239010141700100,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3495,P20239010141700101,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
uid_list = list(interaction_matrix.index)
user_dict = dict()
counter = 0
for i in uid_list:
  user_dict[i] = counter
  counter += 1

In [49]:
def train_test_split(interaction_matrix):
  train = interaction_matrix.copy()
  test = interaction_matrix.copy()
  test.iloc[:,:] = 0
  for user in interaction_matrix.index:
    user_interactions = interaction_matrix.loc[user]
    last_interaction = user_interactions[user_interactions > 0].index[-1]
    test.loc[user, last_interaction] = interaction_matrix.loc[user, last_interaction]
    train.loc[user, last_interaction] = 0
  train_csr = csr_matrix(train.values)
  test_csr = csr_matrix(test.values)
  return train_csr, test_csr

In [50]:
train_set, test_set = train_test_split(interaction_matrix)

#Training e valutazione modello LightFM

In [51]:
model = LightFM(loss="warp",
                random_state = 999,
                learning_rate= 0.001,
                no_components=20,
                user_alpha = 0.0001,
                item_alpha = 0.0001)
model.fit(train_set, epochs = 200, num_threads = 16, verbose= True)

Epoch: 100%|██████████| 200/200 [00:03<00:00, 65.92it/s]


In [83]:
def user_recommend(model, interactions, uid, user_dict, item_dict, threshold = 0.4, k = 5):
  n_users, n_items = interactions.shape
  user = user_dict[uid]
  predictions = model.predict(user, np.arange(n_items), item_features = items_selected_csr)
  scores = pd.Series(predictions)
  scores.index = interactions.columns
  scores = list(pd.Series(scores.sort_values(ascending=False).index))
  known_items = list(pd.Series(interactions.loc[uid,:][interactions.loc[uid] > threshold].index).sort_values(ascending = False))
  scores = [x for x in scores if x not in known_items and x[1:5]=='2023'][0:k]
  print(f"User {uid} bought:")
  counter = 1
  for line in known_items:
    print(f"{counter} - {line}")
    counter += 1
    if counter == 10:
      break
  print(f"User {uid} recommendations:")
  counter = 1
  for line in scores:
    print(f"{counter} - {line}")
    counter += 1


In [84]:
user_recommend(model, interaction_matrix, 91311, user_dict, item_dict)

User 91311 bought:
1 - P20236013374002001
2 - P20235516718190001
3 - P20225713621810040
4 - P20225712820607598
5 - P20225516718190266
6 - A20225516714290140
7 - A20215810122793607
8 - A20215810122793012
User 91311 recommendations:
1 - P20237621850002001
2 - P20236117750001001
3 - P20235813618120001
4 - P20236011966939001
5 - P20236010374060001


In [53]:
precision_train = precision_at_k(model, train_set, k=10).mean()
precision_test = precision_at_k(model, test_set, k=10).mean()

In [54]:
recall_train = recall_at_k(model, train_set, k=10).mean()
recall_test = recall_at_k(model, test_set, k=10).mean()

In [55]:
auc_train = auc_score(model, train_set).mean()
auc_test = auc_score(model, test_set).mean()

In [56]:
print(f"Precision Train: {precision_train}")
print(f"Precision Test: {precision_test}")
print(f"Recall Train: {recall_train}")
print(f"Recall Test: {recall_test}")
print(f"AUC Train: {auc_train}")
print(f"AUC Test: {auc_test}")

Precision Train: 0.016167664900422096
Precision Test: 0.0020587113685905933
Recall Train: 0.05049792693911093
Recall Test: 0.020587113991612657
AUC Train: 0.8015697598457336
AUC Test: 0.6394129395484924
